In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import gc


split train set and test set

In [21]:
data_tick = pd.read_hdf('../data/data_tick_2months.h5', key='tick_data')
data_stock = data_tick.xs('688303.XSHG', level='order_book_id') #! 取第x只股票
date_list = data_stock.trading_date.unique() # 取出所有交易日的date

df_tick_test = data_stock[data_stock['trading_date'] == date_list[-1]]  # 取最后一天数据为测试集
df_tick_train = data_stock[data_stock['trading_date'] != date_list[-1]] # 其他交易日数据为测试集

data_day = pd.read_hdf('../data/data_20days.h5', key='day_data')
data_stock = data_day.xs('688303.XSHG', level='order_book_id') #! 取第x只股票
day_list = data_stock.index.unique() # 取出所有交易日的date

# df_day_test = data_stock[(data_stock['date'] >= day_list[-21]) & (data_stock['date'] < day_list[-1])]  
# df_day_train = data_stock[data_stock['date'] < day_list[-2]] 
df_day_test = data_stock.loc[day_list[-21]: day_list[-1]]
df_day_train = data_stock.loc[: day_list[-2]]

resample tick data in 3s pieces

In [24]:
df_tick_list = []
for i in range(len(date_list)-1):
    data_piece = df_tick_train[df_tick_train['trading_date'] == date_list[i]]
    data_piece = data_piece.resample('3S', origin=f'{date_list[i].date()} 09:30:00').ffill()
    locs1 = data_piece.index.indexer_between_time('09:30:00', '11:30:00', include_start=True,include_end=True)
    locs2 = data_piece.index.indexer_between_time('13:00:00', '15:00:00', include_start=True,include_end=True)
    data_piece = pd.concat((data_piece.iloc[locs1], data_piece.iloc[locs2]), axis=0)
    data_piece[['volume', 'total_turnover', 'num_trades']] = data_piece[['volume', 'total_turnover', 'num_trades']].diff().bfill()
    df_tick_list.append(data_piece)
df_tick_train = pd.concat(df_tick_list, axis=0)

df_tick_test = df_tick_test.resample('3S', origin=f'{date_list[-1].date()} 09:30:00').ffill()
locs1 = df_tick_test.index.indexer_between_time('09:30:00', '11:30:00', include_start=True,include_end=True)
locs2 = df_tick_test.index.indexer_between_time('13:00:00', '15:00:00', include_start=True,include_end=True)
df_tick_test = pd.concat((df_tick_test.iloc[locs1], df_tick_test.iloc[locs2]), axis=0)
df_tick_test[['volume', 'total_turnover', 'num_trades']] = df_tick_test[['volume', 'total_turnover', 'num_trades']].diff().bfill()

gc.collect()

0

Dataset and DataLoader

In [ ]:
class Dataset_train(Dataset):
    def __init__(self, df_tick_train, df_day_train, time_steps):
        self.time_steps = time_steps
        self.df_tick = df_tick_train
        self.df_day = df_day_train

    def __getitem__(self, index):
        pass

    def __len__(self):
        pass


def LoadData():
    pass

In [ ]:
class Dataset_test(Dataset):
    def __init__(self, df_tick_test, df_day, time_steps, stock_code: str):
        self.time_steps = time_steps
        

    def __getitem__(self, index):
        pass

    def __len__(self):
        pass


def LoadData():
    pass

In [10]:
data_days = pd.read_hdf('../data/data_20days.h5', key='day_data')

In [18]:
data_day = data_days.xs('688303.XSHG', level='order_book_id') 
data_day


,prev_close,close,low,limit_up,open,high,total_turnover,volume,limit_down,num_trades
date,,,,,,,,,,
2023-03-31,48.34,45.0114,44.6226,53.6934,44.6874,45.5019,387879191.0,7957285.0,35.7925,12130.0
2023-04-03,48.63,45.1780,44.4468,54.0174,44.9651,45.2428,462603220.0,9533293.0,36.0054,14411.0
2023-04-04,48.81,44.5393,44.4468,54.2117,45.1039,45.1039,457218855.0,9478052.0,36.1442,15193.0
2023-04-06,48.12,44.6782,44.1506,53.4435,44.6504,44.7892,448487958.0,9336875.0,35.6352,14805.0
2023-04-07,48.27,44.7707,44.5208,53.6101,44.5949,44.8818,318355742.0,6586650.0,35.7462,10267.0
...,...,...,...,...,...,...,...,...,...,...
2023-06-27,40.08,40.2100,39.8500,48.1000,40.0000,40.8000,185225989.0,4585645.0,32.0600,7856.0
2023-06-28,40.21,40.4100,39.9100,48.2500,40.2300,40.5900,216650804.0,5382024.0,32.1700,9310.0
2023-06-29,40.41,40.3000,40.1500,48.4900,40.3000,40.6700,151712941.0,3754730.0,32.3300,6339.0
